In [1]:
import pandas as pd

pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 150)

In [2]:
import os
from functools import reduce, partial
from utils import read_yaml
from pathlib import Path
import usuarios
import backup
import vpn_connection as vpn
import usuarios
import acidentes
import acidentes_filtrar
import email_sender

root_dir = Path().resolve().parent

# Importa configurações do sistema
cfg = read_yaml(root_dir / 'config/config.yaml')
secrets = read_yaml(root_dir / 'config/secrets.yaml')
codigos_desativados = read_yaml(root_dir / 'config/codigos_desativados_conversao.yaml')
fatores_params = read_yaml(root_dir / 'config/fatores_risco_classificacao.yaml')
fatores_params_reshaped = acidentes.reshape_fatores_params(fatores_params)

# Indica diretórios e arquivos a serem utilizados
cat_html_template = Path('../data/input/html_templates/cat.html')
alerta_user_html_template = Path('../data/input/html_templates/alerta_usuario.html')
logo_sit = Path('../data/input/images/logoSIT.png')
logo_saat = Path('../data/input/images/logoSAAT_email.png')
output_dir_cat_pdf = Path('../data/output/cat_pdf')
log_dir = Path('../data/log')

# Importa dados que populam as opções do formulário de inscrição
opcoes = usuarios.import_google_spreadsheet(cfg['FORM_INSC_OPCOES']['ID_GSHEET'])

# Importa listas de inscrições e cancelamentos e compila lista de usuários ativos
inscricoes = usuarios.import_google_spreadsheet(cfg['FORM_INSC']['ID_GSHEET'])
cancelamentos = usuarios.import_google_spreadsheet(cfg['FORM_CANCEL']['ID_GSHEET'])
inscricoes_compilado = usuarios.compila_inscricoes(df_inscricao=inscricoes, df_cancelamento=cancelamentos)
inscricoes_compilado_novos_codigos = usuarios.update_codigos_desativados(inscricoes_compilado, codigos_desativados)

# Realiza backup dos dados provenientes dos formulários de inscrição e cancelamento
backup_dir = str(root_dir / 'data/backup')

backup.backup_csv_new_file(opcoes, directory=backup_dir, filename='opcoes_form_insc.csv')
backup.backup_csv(inscricoes, backup_path=os.path.join(backup_dir, 'inscricoes.csv'))
backup.backup_csv(cancelamentos, backup_path=os.path.join(backup_dir, 'cancelamentos.csv'))
backup.backup_csv(inscricoes_compilado_novos_codigos, backup_path=os.path.join(backup_dir, 'inscricoes_compilado.csv'))

Reading configuration file: C:\Users\joao.reis\Pywinauto recorder\config.ini
Window filtering mode: ignore_windows
Ignored windows: []


In [3]:
# Tenta conectar à VPN
vpn.try_connection_forticlient_vpn(vpn_path=cfg['VNP_PATH'],
                                    user=secrets['USER'],
                                    password=secrets['PASSWORD'],
                                    url_test_connection=cfg['VPN_URL_TEST_CONNECTION'])

# Importa novas CATs
cats = acidentes.cat_extrair()

In [4]:
# Trata CATs
cat_atribui_fatores_risco_partial = partial(acidentes.cat_atribui_fatores_risco,
                                            fatores_params_reshaped=fatores_params_reshaped)

functions_list = [acidentes.cat_eliminar_ja_alertadas,
                    acidentes.cat_converter_inteiros,
                    acidentes.cat_converter_datas,
                    acidentes.cat_formatar_horas,
                    acidentes.cat_formatar_strings,
                    acidentes.cat_cid_uppercase,
                    acidentes.cat_novas_colunas,
                    acidentes.cat_uorg_local_acidente,
                    acidentes.cat_secao_cnae_local_acidente,
                    acidentes.cat_formatar_datas,
                    acidentes.cat_identifica_recibo_raiz,
                    acidentes.cat_mantem_recibo_ultima_reabertura,
                    cat_atribui_fatores_risco_partial,
                    acidentes.cat_compila_fatores_risco,
                    acidentes.cat_atribui_consequencia,
                    acidentes.cat_inserir_descricoes
                    ]

cats_tratadas = reduce(lambda x, y: y(x), functions_list, cats)

In [5]:
destinatario = inscricoes_compilado_novos_codigos.iloc[0]

In [6]:
funcoes_filtra_cats = [acidentes_filtrar.uf,
                       acidentes_filtrar.uorg,
                       acidentes_filtrar.tpacid,
                       acidentes_filtrar.consequencias,
                       acidentes_filtrar.risco,
                       acidentes_filtrar.cnae,]

funcoes_filtra_cats_partial = [partial(function, usuario=destinatario) for function in funcoes_filtra_cats]

cats_filtradas = reduce(lambda x, y: y(x), funcoes_filtra_cats_partial, cats_tratadas)

In [7]:
for cat in cats_filtradas.index:
    acidentes.cat_to_pdf(cats_filtradas.loc[cat], 
                         html_template=cat_html_template,
                         logo=logo_sit,
                         output_dir=output_dir_cat_pdf)

In [67]:
def cat_tabela_resumo(df_cat):
    alerta_cols = ['meta_nr_recibo', 'Consequencia', 'CDFatorAmbiental', 'ds_municipio_local_acidente', 'sguf_local_acidente',]
    df = df_cat[alerta_cols].copy()

    df.Consequencia = df.Consequencia.apply(lambda x: '<br>'.join(x))
            
    def map_fr(lista):
        map_fr = pd.read_csv(f'../data/input/aux_tables/fator_risco.csv', dtype='object').set_index('CDFatorAmbiental')['DSFatorAmbiental'].to_dict()
        lista_mapped = [f'{elemento} - {map_fr[elemento]}' for elemento in lista]
        return  '<br>'.join(lista_mapped)
    df['CDFatorAmbiental'] = df['CDFatorAmbiental'].apply(map_fr)
    
    df["Local do Acidente"] = df["ds_municipio_local_acidente"] + '/' + df["sguf_local_acidente"]
    
    df = (df
          .drop(columns=['ds_municipio_local_acidente', 'sguf_local_acidente'])
          .rename(columns={'meta_nr_recibo': 'Número da CAT',
                       'Consequencia': 'Consequências',
                       'CDFatorAmbiental': 'Fatores de risco'}))
    
    return df

In [115]:
teste_html_template = Path('../data/input/html_templates/alerta_usuario.html')

anexos = [output_dir_cat_pdf / f'{cat.meta_nr_recibo}.pdf' for cat in cats_filtradas.itertuples()]

cats_resumo_html = cat_tabela_resumo(cats_filtradas).to_html(index=False, escape=False)

campos = {'cats': cats_resumo_html,
          'uf': destinatario.fillna('-')['UF'],
          'uorg': destinatario.fillna('-')['UORG'],
          'tpacid': destinatario.fillna('-')['Tipo de acidente'],
          'consequencia': destinatario.fillna('-')['Consequência do acidente'],
          'setores': destinatario.fillna('-')['Seção CNAE'],
          'riscos': destinatario.fillna('-')['Fatores de risco']}

msg1 = email_sender.EmailMessageHTML(destinatario=destinatario['E-mail'],
                                     sender_email=cfg['SENDER_EMAIL'],
                                     assunto='Alerta de acidente de trabalho',
                                     template_html= teste_html_template,
                                     template_campos=campos,
                                     anexos=anexos,
                                     imagens=logo_saat)

msg1.send(auth_user=secrets['EMAIL'],
          password=secrets['PASSWORD'],
          smtp_server=cfg['SMPT_SERVER'],
          port=cfg['PORT'])

In [112]:
inscricoes_compilado_novos_codigos['uf_uorg'] = inscricoes_compilado_novos_codigos.UORG.str.extract(r'^([A-Z]{2})\s-')
inscricoes_compilado_novos_codigos['uf_compilado'] =inscricoes_compilado_novos_codigos.UF.fillna(inscricoes_compilado_novos_codigos.uf_uorg)
resumo_inscricoes = (inscricoes_compilado_novos_codigos[['uf_compilado', 'E-mail', 'UORG']]
                     .fillna('Todas')
                     .sort_values(['uf_compilado', 'UORG', 'E-mail',])
                     .rename(columns={'uf_compilado': 'UF'})
                     )

In [113]:
resumo_inscricoes

,UF,E-mail,UORG
3,GO,rosita.dantas@economia.gov.br,Todas
2,MG,marcelo.simeao@economia.gov.br,MG - 021904004 - AGÊNCIA DE ATENDIMENTO DE JOAO MONLEVADE/MG
0,MG,joao.reis@economia.gov.br,Todas
1,SP,julio.cesar@economia.gov.br,Todas


In [121]:
log_dict = {'Coluna A': 'Valor colA',
            'Coluna B': 'Valor colB',
            'Coluna C': 'Valor colC',}
[i for i in log_dict.keys()]
','.join(log_dict.values()) + "\n"

'Valor colA,Valor colB,Valor colC\n'

In [ ]:
def append_to_csv(file: str|Path, 
                  new_line_data: dict):
    
    cabecalho = ','.join(f'"{w}"' for w in new_line_data.keys()) + "\n"
    linha = ','.join(f'"{w}"' for w in new_line_data.values()) + "\n"

    if not os.path.exists(file):
        with open(file, mode='w') as csv_file:
            csv_file.write(cabecalho + linha)

    else:
        with open(file) as csv_file:
            cabecalho_atual = csv_file.readline()
            if cabecalho_atual != cabecalho:
                raise ValueError('As colunas não correspondem ao cabeçalho do arquivo indicado.')

        with open(file, 'a') as csv_file:
            csv_file.write(linha)

log_file = 'teste.csv'

log_dict = {'Coluna A': 'Valor colA',
            'Coluna B': 'Valor colB',
            'Coluna C': 'Valor colC',}

append_to_csv(log_file, log_dict)